In [ ]:
#I executed these commands in the terminal.”

# Loading packages
brew install fastqc
brew install fastp
brew install bwa
brew install samtools
brew install samtools


mkdir -p ~/gtmolecular_project/{fastq,qc,trimmed,alignments,results,logs}

mv ~/Downloads/SRR13974119.fastq.gz ~/gtmolecular_project/fastq/ #Pre-treatment
mv ~/Downloads/SRR13974120.fastq.gz ~/gtmolecular_project/fastq/ #Post-treatment


#Generating QC report
fastqc ~/gtmolecular_project/fastq/*.fastq.gz -o ~/gtmolecular_project/qc


# Trimming with fastp

##SRR13974119
fastp -i ~/gtmolecular_project/fastq/SRR13974119.fastq.gz \
      -o ~/gtmolecular_project/trimmed/SRR13974119_trimmed.fastq.gz \
      --detect_adapter_for_pe \
      --html ~/gtmolecular_project/trimmed/SRR13974119_fastp_report.html \
      --thread 4

##SRR13974120
fastp -i ~/gtmolecular_project/fastq/SRR13974120.fastq.gz \
      -o ~/gtmolecular_project/trimmed/SRR13974120_trimmed.fastq.gz \
      --detect_adapter_for_pe \
      --html ~/gtmolecular_project/trimmed/SRR13974120_fastp_report.html \
      --thread 4



mkdir -p ~/genomes/hg38
cd ~/genomes/hg38

#Loading Reference genome
curl -O ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_43/GRCh38.primary_assembly.genome.fa.gz


gunzip GRCh38.primary_assembly.genome.fa.gz

#Indexing
bwa index ~/genomes/hg38/GRCh38.primary_assembly.genome.fa


#Alignment with bwa
##SRR13974119
bwa mem -t 6 ~/genomes/hg38/GRCh38.primary_assembly.genome.fa \
    ~/gtmolecular_project/trimmed/SRR13974119_trimmed.fastq.gz \
  | samtools view -bS - \
  | samtools sort -@ 4 -o ~/gtmolecular_project/alignments/SRR13974119_sorted.bam

samtools index ~/gtmolecular_project/alignments/SRR13974119_sorted.bam


##SRR13974120
bwa mem -t 6 ~/genomes/hg38/GRCh38.primary_assembly.genome.fa \
    ~/gtmolecular_project/trimmed/SRR13974120_trimmed.fastq.gz \
  | samtools view -bS - \
  | samtools sort -@ 4 -o ~/gtmolecular_project/alignments/SRR13974120_sorted.bam

samtools index ~/gtmolecular_project/alignments/SRR13974120_sorted.bam


#Marking Duplicates
##SRR13974119
samtools markdup -r \
  ~/gtmolecular_project/alignments/SRR13974119_sorted.bam \
  ~/gtmolecular_project/alignments/SRR13974119_dedup.bam

samtools index ~/gtmolecular_project/alignments/SRR13974119_dedup.bam

##SRR13974120
samtools markdup -r \
  ~/gtmolecular_project/alignments/SRR13974120_sorted.bam \
  ~/gtmolecular_project/alignments/SRR13974120_dedup.bam

samtools index ~/gtmolecular_project/alignments/SRR13974120_dedup.bam


#Loading PBMC FASTQ file (Normal)(SRR13973869)
mv ~/Downloads/SRR13973869.fastq.gz ~/gtmolecular_project/fastq/

fastqc ~/gtmolecular_project/fastq/SRR13973869.fastq.gz -o ~/gtmolecular_project/qc

fastp -i ~/gtmolecular_project/fastq/SRR13973869.fastq.gz \
      -o ~/gtmolecular_project/trimmed/SRR13973869_trimmed.fastq.gz \
      --detect_adapter_for_pe \
      --html ~/gtmolecular_project/trimmed/SRR13973869_fastp_report.html \
      --thread 4

bwa mem -t 6 ~/genomes/hg38/GRCh38.primary_assembly.genome.fa \
    ~/gtmolecular_project/trimmed/SRR13973869_trimmed.fastq.gz \
  | samtools view -bS - \
  | samtools sort -@ 4 -o ~/gtmolecular_project/alignments/SRR13973869_sorted.bam

samtools index ~/gtmolecular_project/alignments/SRR13973869_sorted.bam

samtools markdup -r \
  ~/gtmolecular_project/alignments/SRR13973869_sorted.bam \
  ~/gtmolecular_project/alignments/SRR13973869_dedup.bam

samtools index ~/gtmolecular_project/alignments/SRR13973869_dedup.bam

#index a reference genome
samtools faidx ~/genomes/hg38/GRCh38.primary_assembly.genome.fa


mkdir -p ~/gtmolecular_project/variants

# Varinat Calling with FreeBayes, PBMC as a normal/control
freebayes -f ~/genomes/hg38/GRCh38.primary_assembly.genome.fa \
  ~/gtmolecular_project/alignments/SRR13974119_dedup.bam \
  ~/gtmolecular_project/alignments/SRR13973869_dedup.bam \
  > ~/gtmolecular_project/variants/SRR13974119_freebayes.vcf

freebayes -f ~/genomes/hg38/GRCh38.primary_assembly.genome.fa \
  ~/gtmolecular_project/alignments/SRR13974120_dedup.bam \
  ~/gtmolecular_project/alignments/SRR13973869_dedup.bam \
  > ~/gtmolecular_project/variants/SRR13974120_freebayes.vcf